In [1]:
# import pandas as pd

# out = pd.read_csv('data/cf_apprissAppend_va2_10_17_2022.csv')

In [2]:
#using python 3.8.10
#imports 
import pandas as pd
from datetime import datetime
import csv
import numpy as np

column_names = ['source_id', "popID"]
#here we are reading in our criminal data
criminal_df = pd.read_csv("data/crime_cut_down_v1.csv")
NCIC_dic_df = pd.read_csv("data/diction_NCICcodes_descriptions_CrimeTiers_CDCTiers_as_csv.csv")

In [3]:
# # before we turn crime file into flat file we need to cut down this list with our new confidence score as some of these people did are false matches to the crime data we received

In [4]:
# remove the '[' and ']' from the NCIC code column
criminal_df['BookingChargeNCICOffenseCode'] = criminal_df['BookingChargeNCICOffenseCode'].str.replace('[', '')
criminal_df['BookingChargeNCICOffenseCode'] = criminal_df['BookingChargeNCICOffenseCode'].str.replace(']', '')

C:\Users\cayde\AppData\Local\Temp\ipykernel_7588\966273755.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  criminal_df['BookingChargeNCICOffenseCode'] = criminal_df['BookingChargeNCICOffenseCode'].str.replace('[', '')
C:\Users\cayde\AppData\Local\Temp\ipykernel_7588\966273755.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  criminal_df['BookingChargeNCICOffenseCode'] = criminal_df['BookingChargeNCICOffenseCode'].str.replace(']', '')


In [6]:
# here we will scan our data and remove any NCIC codes where we have no examples in the entire dataframe
### this cuts down on the dimensions of the data frame from 1600x455 to 1600x52
# here we are creating a subset of the raw data that has the CorrelationID, BookingDate, and unrolled NCICcodes per row
# each rows unrolled NCIC offense codes
unrolled_NCICCodes_df = criminal_df['BookingChargeNCICOffenseCode'].str.split(', ', expand=True)

#pull the correlationId and bookingdate columns for the data set and join them to the unrolled NCIC codes
unrolled_NCICCodes_df.insert(0, 'BookingDate', criminal_df['BookingDate'])
unrolled_NCICCodes_df.insert(0, 'source_id', criminal_df['source_id'])
unrolled_NCICCodes_df = unrolled_NCICCodes_df.fillna('')

# here we are gonna make a distrobustion of counts of NCIC codes
column_names_1 = ['NCICcode_count']
index_names = []
NCICcode_counts_df = pd.DataFrame(columns = column_names_1,)

for index, row in NCIC_dic_df.iterrows():
    if(row['NCIC Offense Code'] is not None and row['NCIC Offense Code'] != ''):
        df = pd.DataFrame({"NCICcode_count":[0]})
        NCICcode_counts_df = pd.concat([NCICcode_counts_df, df])
        index_names.append(row['NCIC Offense Code'])

NCICcode_counts_df = pd.DataFrame(NCICcode_counts_df).set_index([index_names])

for ind, row in unrolled_NCICCodes_df.iterrows():
    #print(row['CorrelationID'], row['BookingDate'], row[0])
    # for i in range(0, 55):
    for i in range(0, 48):
        if(row[i] is not None and row[i] != ''):
            NCICcode = int(row[i])
            NCICcode_counts_df.at[NCICcode, 'NCICcode_count'] = NCICcode_counts_df.at[NCICcode, 'NCICcode_count'] + 1

no_examples = []
counter = 0
for ind, row in NCICcode_counts_df.iterrows():
    if(row['NCICcode_count'] == 0):
        no_examples.append(row.name)
    else:
        counter = counter + 1

print("455 ncic codes")
print('NCIC codes that never show up in dataset: ' + str(len(no_examples)))
print('NCIC codes that show up in dataset: ' + str(counter))

455 ncic codes
NCIC codes that never show up in dataset: 405
NCIC codes that show up in dataset: 50


In [7]:
#here we are making the crime summary data frame
for index, row in NCIC_dic_df.iterrows():
    if(row['NCIC Offense Code'] not in no_examples):
        if(row['NCIC Offense Code'] is not None and row['NCIC Offense Code'] != ''):
            column_names.append(row['NCIC Offense Code'])

criminalData_summary_df = pd.DataFrame(columns = column_names)
unique_ids_column = criminal_df['source_id'].unique()


for index in unique_ids_column:
    df = pd.DataFrame({"source_id":[index]})
    criminalData_summary_df = pd.concat([criminalData_summary_df, df])

criminalData_summary_df = criminalData_summary_df.fillna(0)
criminalData_summary_df.reset_index(drop=True, inplace=True)

# here we are adding a new columns for crimeTiers and CDCcats
criminalData_summary_df = criminalData_summary_df.assign(tier_1=0)
criminalData_summary_df = criminalData_summary_df.assign(tier_2=0)
criminalData_summary_df = criminalData_summary_df.assign(tier_3=0)
criminalData_summary_df = criminalData_summary_df.assign(tier_4=0)

criminalData_summary_df = criminalData_summary_df.assign(CDCcat_A=0)
criminalData_summary_df = criminalData_summary_df.assign(CDCcat_B=0)
criminalData_summary_df = criminalData_summary_df.assign(CDCcat_C=0)
criminalData_summary_df = criminalData_summary_df.assign(CDCcat_D=0)
criminalData_summary_df = criminalData_summary_df.assign(CDCcat_E=0)

# # here we are creating a subset of the raw data that has the CorrelationID, BookingDate, and unrolled NCICcodes per row
# # each rows unrolled NCIC offense codes
# unrolled_NCICCodes_df = criminal_df['BookingChargeNCICOffenseCode'].str.split(', ', expand=True)

# #pull the correlationId and bookingdate columns for the data set and join them to the unrolled NCIC codes
# unrolled_NCICCodes_df.insert(0, 'BookingDate', criminal_df['BookingDate'])
# unrolled_NCICCodes_df.insert(0, 'CorrelationID', criminal_df['CorrelationID'])
# unrolled_NCICCodes_df = unrolled_NCICCodes_df.fillna('')



In [8]:
# #here we build the dataset for alltime
# master_dataset = criminalData_summary_df

#This method is a simple check for handling tiers and crime cats. so gross ik but it works
def check(val):
    if(val == 1.0):
        return "tier_1"
    if(val == 2.0):
        return "tier_2"
    if(val == 3.0):
        return "tier_3"
    if(val == 4.0):
        return "tier_4"
    if(val == 'A'):
        return "CDCcat_A"
    if(val == 'B'):
        return "CDCcat_B"
    if(val == 'C'):
        return "CDCcat_C"
    if(val == 'D'):
        return "CDCcat_D"
    if(val == 'E'):
        return "CDCcat_E"
    if(val == None):
        return None
    if(val == 'NaN'):
        return None


# for ind, row in unrolled_NCICCodes_df.iterrows():
#     for i in range(0, 55):
#             if(row[i] is not None and row[i] != ''):
#                 NCICcode = int(row[i])
#                 summary_row = master_dataset[master_dataset['CorrelationID'] == row['CorrelationID']]
#                 master_dataset.at[summary_row.index[0], NCICcode] = master_dataset.at[summary_row.index[0], NCICcode] +1
#                 #this lower part handles both tiers and cdccat
#                 TierVal = dict_df.at[NCICcode, "Tier"]
#                 CDCcat_val = dict_df.at[NCICcode, "CDC Category"]
#                 if(check(TierVal) != None):
#                     master_dataset.at[summary_row.index[0], check(TierVal)] = master_dataset.at[summary_row.index[0], check(TierVal)] +1
#                 if(check(CDCcat_val) != None):
#                     master_dataset.at[summary_row.index[0], check(CDCcat_val)] = master_dataset.at[summary_row.index[0], check(CDCcat_val)] +1

                

In [9]:
# what are the dtype for the columns of the dataframe NCIC_dic_df
print(NCIC_dic_df.dtypes)

# make NCIC Offense Code a the index of the dataframe
NCIC_dic_df = NCIC_dic_df.set_index('NCIC Offense Code')

NCIC Offense Code         int64
Description of Crime     object
Notes:                   object
Tier                    float64
CDC Category             object
dtype: object


In [10]:
#this is where we actually genertae the values and put them into the dataset
endDate_alltime = datetime(month=9,day=14,year=2021) # this is the last day of t0 for population 4

#here we build the dataset for alltime
master_dataset = criminalData_summary_df
for ind, row in unrolled_NCICCodes_df.iterrows():
    # for i in range(0, 55):
    for i in range(0, 48):
            if(row[i] is not None and row[i] != ''):
                NCICcode = int(row[i])
                summary_row = master_dataset[master_dataset['source_id'] == row['source_id']]
                master_dataset.at[summary_row.index[0], NCICcode] = master_dataset.at[summary_row.index[0], NCICcode] +1
                #this lower part handles both tiers and cdccat
                TierVal = NCIC_dic_df.at[NCICcode, "Tier"]
                CDCcat_val = NCIC_dic_df.at[NCICcode, "CDC Category"]
                if(check(TierVal) != None):
                    master_dataset.at[summary_row.index[0], check(TierVal)] = master_dataset.at[summary_row.index[0], check(TierVal)] +1
                if(check(CDCcat_val) != None):
                    master_dataset.at[summary_row.index[0], check(CDCcat_val)] = master_dataset.at[summary_row.index[0], check(CDCcat_val)] +1

criminalData_summary_df = criminalData_summary_df.drop(['popID'],axis=1)

# # # we need to build 7 datasets which are ALLtime and t0-t6
# for j in range(0,7): # needs to be 0-6 = 7 but start with 0-1 =2. NOTE increasing this to 9 to cover t_mb6 and t_mb12
#     dataset = criminalData_summary_df.copy()
#     for ind, row in unrolled_NCICCodes_df.iterrows():
#         BookingDate = datetime.strptime(row['BookingDate'], '%m/%d/%Y') # here is the booking date for each row
#         if(isinstance(row['deathDate'], float)): #did not kill self
#             endDate = endDate_alltime - pd.DateOffset(months=6 * j)
#             startDate = endDate_alltime - pd.DateOffset(months=6 * (j + 1))
#             # print('alive')
#             # print(endDate)
#             # print(startDate)
#         else: # killed self
#             deathDate = datetime.strptime(row['deathDate'], '%m/%d/%Y')
#             endDate = deathDate - pd.DateOffset(months=6 * j)
#             startDate = deathDate - pd.DateOffset(months=6 * (j + 1))
#             # print("dead")
#             # print(row['deathDate'])
#             # print(endDate)
#             # print(startDate)
#         if(j > 6):
#             for i in range(0, 55):
#                 if(row[i] is not None and row[i] != ''): # here is the case where we are in t0-t6 time
#                     if(startDate <= BookingDate <= endDate):
#                         NCICcode = int(row[i])
#                         summary_row = dataset[dataset['CorrelationID'] == row['CorrelationID']]
#                         dataset.at[summary_row.index[0], NCICcode] = dataset.at[summary_row.index[0], NCICcode] +1
#                         #this lower part handles both tiers and cdccat
#                         TierVal = dict_df.at[NCICcode, "Tier"]
#                         CDCcat_val = dict_df.at[NCICcode, "CDC Category"]
#                         if(check(TierVal) != None):
#                             master_dataset.at[summary_row.index[0], check(TierVal)] = master_dataset.at[summary_row.index[0], check(TierVal)] +1
#                         if(check(CDCcat_val) != None):
#                             master_dataset.at[summary_row.index[0], check(CDCcat_val)] = master_dataset.at[summary_row.index[0], check(CDCcat_val)] +1
#         if(j == 6):
#             for i in range(0, 55):
#                 if(row[i] is not None and row[i] != ''): # here is the case where we are in t0-t6 time
#                     if(BookingDate <= endDate):
#                         NCICcode = int(row[i])
#                         summary_row = dataset[dataset['CorrelationID'] == row['CorrelationID']]
#                         dataset.at[summary_row.index[0], NCICcode] = dataset.at[summary_row.index[0], NCICcode] +1
#                         #this lower part handles both tiers and cdccat
#                         TierVal = dict_df.at[NCICcode, "Tier"]
#                         CDCcat_val = dict_df.at[NCICcode, "CDC Category"]
#                         if(check(TierVal) != None):
#                             master_dataset.at[summary_row.index[0], check(TierVal)] = master_dataset.at[summary_row.index[0], check(TierVal)] +1
#                         if(check(CDCcat_val) != None):
#                             master_dataset.at[summary_row.index[0], check(CDCcat_val)] = master_dataset.at[summary_row.index[0], check(CDCcat_val)] +1
#     for col in dataset.columns:
#         new_string = str(col) + "_" + "t" + str(j)
#         if(col != "CorrelationID"):
#             dataset.rename(columns={col:new_string},inplace=True)
#     master_dataset = pd.merge(master_dataset, dataset, left_on= "CorrelationID", right_on='CorrelationID', how="outer")

# for j in range(1,3): # NOTE this is to cover t_mb6 and t_mb12
#     dataset = criminalData_summary_df.copy()
#     for ind, row in unrolled_NCICCodes_df.iterrows():
#         BookingDate = datetime.strptime(row['BookingDate'], '%m/%d/%Y') # here is the booking date for each row
#         if(isinstance(row['deathDate'], float)): #did not kill self
#             endDate = endDate_alltime # death date does not shift in this one
#             startDate = endDate_alltime - pd.DateOffset(months=6 * j)
#         else: # killed self
#             deathDate = datetime.strptime(row['deathDate'], '%m/%d/%Y')
#             endDate = deathDate # death date does not shift in this one
#             startDate = deathDate - pd.DateOffset(months=6 * j)
#         for i in range(0, 55):
#                 if(row[i] is not None and row[i] != ''): # here is the case where we are in t0-t6 time
#                     if(BookingDate <= endDate):
#                         NCICcode = int(row[i])
#                         summary_row = dataset[dataset['CorrelationID'] == row['CorrelationID']]
#                         dataset.at[summary_row.index[0], NCICcode] = dataset.at[summary_row.index[0], NCICcode] +1
#                         #this lower part handles both tiers and cdccat
#                         TierVal = dict_df.at[NCICcode, "Tier"]
#                         CDCcat_val = dict_df.at[NCICcode, "CDC Category"]
#                         if(check(TierVal) != None):
#                             master_dataset.at[summary_row.index[0], check(TierVal)] = master_dataset.at[summary_row.index[0], check(TierVal)] +1
#                         if(check(CDCcat_val) != None):
#                             master_dataset.at[summary_row.index[0], check(CDCcat_val)] = master_dataset.at[summary_row.index[0], check(CDCcat_val)] +1
#     for col in dataset.columns:
#         new_string = str(col) + "_" + "t_" + str(j*6) + "mb"
#         if(col != "CorrelationID"):
#             dataset.rename(columns={col:new_string},inplace=True)
#     master_dataset = pd.merge(master_dataset, dataset, left_on= "CorrelationID", right_on='CorrelationID', how="outer")

#here is how we fill in pop id
#master_dataset.at[summary_row.index[0], 'popID'] = int(row['popID'])

In [11]:
# here we are adding a new column to the end of the dataframe. this is all the crime data we have. Therefore each person in this summary has a criminal record. this is why we assign a value of 1 here. 
master_dataset = master_dataset.assign(HasCriminalRecord=1)

# drop popID column
master_dataset = master_dataset.drop(columns=['popID'])

In [12]:
# for every column in the dataframe, if the value of each row is non zero create a new column with the same name but with a 1 in it. this is to indicate that the person has a criminal record for that crime.
for col in master_dataset.columns:
    if(col != "source_id" and col != "HasCriminalRecord"):
        master_dataset = master_dataset.assign(**{str(col) + "_HasCrime": master_dataset[col].apply(lambda x: 1 if x != 0 else 0)})


In [13]:
# write to csv
master_dataset.to_csv("data/out/appriss_crime_cleaned_CFconfidenceapplied_v2.csv", index=False)